## Downloading and preparing PubMed_QA dataset

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 8.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

datasets = load_dataset("pubmed_qa", "pqa_artificial") # subsets: pqa_labeled, pqa_unlabeled, pqa_artificial

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/211269 [00:00<?, ? examples/s]

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['pubid', 'question', 'context', 'long_answer', 'final_decision'],
        num_rows: 211269
    })
})

In [ ]:
for split, dataset in datasets.items():
  dataset.to_csv(f"pubmed_qa_artificial_{split}.csv", index = None)

Creating CSV from Arrow format:   0%|          | 0/212 [00:00<?, ?ba/s]

### Extracting pubmed_qa article abstracts


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
import json
import numpy as np

In [ ]:
# loading csv files
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
download = drive.CreateFile({'id': '1vx89vDIvGtc5w_kGp_T2OtCYDSnvhAEn'})
download.GetContentFile('pubmed_qa_labeled_train.csv')

download = drive.CreateFile({'id': '1oIx_jUKPhgYro1d-hab4XH1y3zx3V5xM'})
download.GetContentFile('pubmed_qa_artificial_train.csv')

download = drive.CreateFile({'id': '1-IcL2llqRqMVc3vMSumzQiHCILYUtcUR'})
download.GetContentFile('pubmed_qa_unlabeled_train.csv')

In [ ]:
files = ['labeled', 'unlabeled', 'artificial']

dfs = []
for file in files:
  dfs.append(pd.read_csv(f"pubmed_qa_{file}_train.csv"))

In [ ]:
for idx, df in enumerate(dfs):
  raw_data = []
  for index, row in df.iterrows():
      pubid = row["pubid"]
      context = row["context"]
      context = context.replace("array(", "np.array(")
      context_dict = eval(context)
      contexts = context_dict["contexts"]
      final_context = ' '.join(contexts)
      raw_data.append([pubid, final_context])
  data = pd.DataFrame(raw_data, columns=['pubid', 'context'])
  data.to_csv(f"pubmed_qa_{files[idx]}_articles.csv", sep='\t')

In [ ]:
df = pd.read_csv('pubmed_qa_artificial_articles.csv', sep='\t')
df

,Unnamed: 0,pubid,context
0,0,25429730,Chronic rhinosinusitis (CRS) is a heterogeneou...
1,1,25433161,Phosphatidylethanolamine N-methyltransferase (...
2,2,25445714,Psammaplin A (PsA) is a natural product isolat...
3,3,25431941,This study examined links between DNA methylat...
4,4,25432519,Tumor microenvironment immunity is associated ...
...,...,...,...
211264,211264,8217974,To investigate the relation between hourly fet...
211265,211265,8204319,This study set out to show how well samples fr...
211266,211266,8205673,There is increasing recognition of myocardial ...
211267,211267,8215873,To determine if splenectomy results in an incr...
